In [10]:
import numpy as np
import os
import sys
import argparse
import glob
import time
sys.path.append("/home/jungmo/apps/caffe/python") # caffe path

import caffe
from shutil import copyfile

#GPUID = 1
TRAIN = "/home/jungmo/Documents/autonomous/caffe/bird_image/train/train.txt"
TEST = "/home/jungmo/Documents/autonomous/caffe/bird_image/test/test_256_200.txt"
BIRD_ROOT = "/home/jungmo/Documents/autonomous/caffe/"

BINARYPROTO_PATH = os.path.join(BIRD_ROOT, "db/mean.binaryproto")
NPY_PATH = os.path.join(BIRD_ROOT, "db/mean.npy")
CAFFE_BIN = "/home/jungmo/apps/caffe/build/tools/"
network = 'googLeNet'
IMG_SIZE = 224

problem = open('egg_classified_as_bird.txt', "w")
def binaryproto_to_npy():
    blob = caffe.proto.caffe_pb2.BlobProto()
    data = open( BINARYPROTO_PATH , 'rb' ).read()
    blob.ParseFromString(data)
    arr = np.array( caffe.io.blobproto_to_array(blob) )
    out = arr[0]
    np.save( NPY_PATH, out )

def evaluate_model(dst):
        input_file = os.path.join(TEST)
        output_file = os.path.join(BIRD_ROOT, "db/output.npy")
        model_def = os.path.join(BIRD_ROOT, "model/"+network+"/deploy.prototxt")
        pretrained_model = os.path.join(BIRD_ROOT, "googlenet_iter_100000.caffemodel")
        gpu = True
        images_dim='224,224'
        mean_file = NPY_PATH
        ext='bmp'
        channel_swap='2,1,0'
        raw_scale = 255.0
        center_only=True
        input_scale = None
        image_dims = [int(s) for s in images_dim.split(',')]

        mean, channel_swap = None, None
        if mean_file:
            mean = np.load(mean_file)
        if channel_swap:
            channel_swap=[int(s) for s in args.channel_swap.split(',')]
        caffe.set_mode_gpu()
        #caffe.set_device(GPUID)
        # Make classifier.
        classifier = caffe.Classifier(model_def, pretrained_model,
                image_dims=image_dims, mean=mean,
                input_scale=input_scale, raw_scale=raw_scale,
                channel_swap=channel_swap)

        # Load numpy array (.npy), directory glob (*.jpg), or image file.
        imgs_list = tuple(open(input_file,'r'))
        print("Loading file: %s" % input_file)
        inputs =[caffe.io.load_image(im_f)
                     for im_f in glob.glob(input_file + '/*.' + ext)]

        print("Classifying %d inputs." % len(imgs_list))

        BATCH_SIZE = 100

        # Classify.
        confusion_matrix = np.zeros((4,4),dtype=np.int32)
        accuracy = 0.0
        start = time.time()
        for i in range(0, len(imgs_list), BATCH_SIZE):
            if i > 0 and i % (BATCH_SIZE * 25) == 0:
                print("Finished %d imgs." % (i))
                cur_accuracy = accuracy / i * 100
                print("Accuracy: %.2f " % cur_accuracy)
                print("Confusion Matrix:")
                print confusion_matrix
            imgs = [imgs_list[i + j].split(' ')[0] for j in range(min(BATCH_SIZE,len(imgs_list)-i))]
            gts = [int(imgs_list[i + j].split(' ')[1]) for j in range(min(BATCH_SIZE,len(imgs_list)-i))]
            inputs = [caffe.io.load_image(path) for path in imgs]
            predictions = classifier.predict(inputs, not center_only)
            for j in range(len(imgs)):
                probs = predictions[j]
                pred = np.argmax(probs)
                confusion_matrix[gts[j]][pred] += 1
                if pred == gts[j]:
                    accuracy += 1.0

        accuracy = accuracy / len(imgs_list) * 100

        print("Done in %.2f s." % (time.time() - start))
        # Save
        print("Accuracy: %.2f " % accuracy)
        print("Confusion Matrix:")
        print confusion_matrix
        np.save(dst, confusion_matrix)

In [11]:

'''    
print "Make DB : Training set"
os.system(os.path.join(CAFFE_BIN,"convert_imageset.bin")+" -backend=\"lmdb\" -shuffle=true -resize_height "+str(IMG_SIZE)+" -resize_width "+str(IMG_SIZE)+ " / " + TRAIN + " " + BIRD_ROOT + "db/train_imageData_lmDB")

print "Make DB : Test set"
os.system(os.path.join(CAFFE_BIN,"convert_imageset.bin")+" -backend=\"lmdb\" -shuffle=true -resize_height "+str(IMG_SIZE)+" -resize_width "+str(IMG_SIZE) + " / " + TEST + " " + BIRD_ROOT + "db/val_imageData_lmDB")

print "Make image mean file"
os.system(os.path.join(CAFFE_BIN,"compute_image_mean.bin")+
               " -backend=lmdb "+ BIRD_ROOT + "db/train_imageData_lmDB " + BIRD_ROOT+ "db/mean.binaryproto")
print "Image mean file .binaryproto to .npy"
binaryproto_to_npy()   

print "Training model"
os.system(os.path.join(CAFFE_BIN,"caffe.bin")+" train "
               "-solver="+BIRD_ROOT+"model/"+network+"/solver.prototxt "+" -weights="+BIRD_ROOT+"model/"+network+"/bvlc_googlenet.caffemodel "+ "-gpu all")
'''
print "Evaluate model"
TEST = "/home/jungmo/Documents/autonomous/caffe/bird_image/test/test_256_200.txt"
evaluate_model("test_256_200.npy")


Evaluate model
Loading file: /home/jungmo/Documents/autonomous/caffe/bird_image/test/test_256_200.txt
Classifying 24163 inputs.
Finished 2500 imgs.
Accuracy: 98.76 
Confusion Matrix:
[[2469    6    7   18]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 98.54 
Confusion Matrix:
[[4927   11   10   52]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 97.71 
Confusion Matrix:
[[6652   14   12   60]
 [  33  676   41   12]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 97.13 
Confusion Matrix:
[[8423   16   35   78]
 [  57 1290   76   25]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 97.02 
Confusion Matrix:
[[10806    20    64   118]
 [   57  1290    76    25]
 [    9     0    32     3]
 [    0     0     0     0]]
Finished 15000 imgs.
Accuracy: 91.65 
Confusion Matrix:
[[10806    20    64   118]
 [   57  1290    76    25]
 [  

In [12]:
TEST = "/home/jungmo/Documents/autonomous/caffe/bird_image/test/test_256_150.txt"
evaluate_model("test_256_150.npy")

Loading file: /home/jungmo/Documents/autonomous/caffe/bird_image/test/test_256_150.txt
Classifying 24163 inputs.
Finished 2500 imgs.
Accuracy: 99.08 
Confusion Matrix:
[[2477    4    6   13]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 99.02 
Confusion Matrix:
[[4951   11    9   29]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 97.83 
Confusion Matrix:
[[6669   14   12   43]
 [  42  668   40   12]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 97.37 
Confusion Matrix:
[[8456   15   21   60]
 [  81 1281   70   16]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 97.46 
Confusion Matrix:
[[10876    20    26    86]
 [   81  1281    70    16]
 [   15     0    25     4]
 [    0     0     0     0]]
Finished 15000 imgs.
Accuracy: 90.49 
Confusion Matrix:
[[10876    20    26    86]
 [   81  1281    70    16]
 [ 1030     2  1417

In [13]:
TEST = "/home/jungmo/Documents/autonomous/caffe/bird_image/test/test_256_140.txt"
evaluate_model("test_256_140.npy")

Loading file: /home/jungmo/Documents/autonomous/caffe/bird_image/test/test_256_140.txt
Classifying 24163 inputs.
Finished 2500 imgs.
Accuracy: 98.88 
Confusion Matrix:
[[2472    6    5   17]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 99.02 
Confusion Matrix:
[[4951   12    8   29]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 97.85 
Confusion Matrix:
[[6666   13   14   45]
 [  50  673   27   12]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 97.21 
Confusion Matrix:
[[8446   15   25   66]
 [  86 1275   69   18]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 97.36 
Confusion Matrix:
[[10869    18    35    86]
 [   86  1275    69    18]
 [   16     0    26     2]
 [    0     0     0     0]]
Finished 15000 imgs.
Accuracy: 90.91 
Confusion Matrix:
[[10869    18    35    86]
 [   86  1275    69    18]
 [  914     5  1493

In [14]:
TEST = "/home/jungmo/Documents/autonomous/caffe/bird_image/test/test_256_130.txt"
evaluate_model("test_256_130.npy")

Loading file: /home/jungmo/Documents/autonomous/caffe/bird_image/test/test_256_130.txt
Classifying 24163 inputs.
Finished 2500 imgs.
Accuracy: 98.76 
Confusion Matrix:
[[2469    8    4   19]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 98.78 
Confusion Matrix:
[[4939   13   14   34]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 97.81 
Confusion Matrix:
[[6657   18   16   47]
 [  37  679   43    3]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 97.18 
Confusion Matrix:
[[8437   20   27   68]
 [  83 1281   75    9]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 97.29 
Confusion Matrix:
[[10856    24    39    89]
 [   83  1281    75     9]
 [   19     0    24     1]
 [    0     0     0     0]]
Finished 15000 imgs.
Accuracy: 89.92 
Confusion Matrix:
[[10856    24    39    89]
 [   83  1281    75     9]
 [ 1109     2  1351

In [15]:
TEST = "/home/jungmo/Documents/autonomous/caffe/bird_image/test/test_256_120.txt"
evaluate_model("test_256_120.npy")

Loading file: /home/jungmo/Documents/autonomous/caffe/bird_image/test/test_256_120.txt
Classifying 24163 inputs.
Finished 2500 imgs.
Accuracy: 98.68 
Confusion Matrix:
[[2467    3   10   20]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 98.58 
Confusion Matrix:
[[4929    7   23   41]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 97.59 
Confusion Matrix:
[[6646   13   26   53]
 [  40  673   39   10]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 96.93 
Confusion Matrix:
[[8423   17   42   70]
 [  75 1270   73   30]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 97.04 
Confusion Matrix:
[[10837    19    53    99]
 [   75  1270    73    30]
 [   20     0    23     1]
 [    0     0     0     0]]
Finished 15000 imgs.
Accuracy: 91.09 
Confusion Matrix:
[[10837    19    53    99]
 [   75  1270    73    30]
 [  909     3  1557

In [16]:
TEST = "/home/jungmo/Documents/autonomous/caffe/bird_image/test/test_256_110.txt"
evaluate_model("test_256_110.npy")

Loading file: /home/jungmo/Documents/autonomous/caffe/bird_image/test/test_256_110.txt
Classifying 24163 inputs.
Finished 2500 imgs.
Accuracy: 98.96 
Confusion Matrix:
[[2474    7    4   15]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 99.06 
Confusion Matrix:
[[4953   13    9   25]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 97.92 
Confusion Matrix:
[[6676   16   12   34]
 [  50  668   36    8]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 97.48 
Confusion Matrix:
[[8466   18   18   50]
 [  95 1282   62    9]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 97.51 
Confusion Matrix:
[[10883    20    21    84]
 [   95  1282    62     9]
 [   20     0    24     0]
 [    0     0     0     0]]
Finished 15000 imgs.
Accuracy: 89.83 
Confusion Matrix:
[[10883    20    21    84]
 [   95  1282    62     9]
 [ 1164     7  1310

In [17]:
TEST = "/home/jungmo/Documents/autonomous/caffe/bird_image/test/test_256_100.txt"
evaluate_model("test_256_100.npy")

Loading file: /home/jungmo/Documents/autonomous/caffe/bird_image/test/test_256_100.txt
Classifying 24163 inputs.
Finished 2500 imgs.
Accuracy: 98.92 
Confusion Matrix:
[[2473   10    3   14]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 98.96 
Confusion Matrix:
[[4948   19    8   25]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 98.12 
Confusion Matrix:
[[6675   21    8   34]
 [  46  684   27    5]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 97.72 
Confusion Matrix:
[[8479   23    8   42]
 [  98 1293   51    6]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 97.83 
Confusion Matrix:
[[10921    25     9    53]
 [   98  1293    51     6]
 [   28     0    15     1]
 [    0     0     0     0]]
Finished 15000 imgs.
Accuracy: 87.77 
Confusion Matrix:
[[10921    25     9    53]
 [   98  1293    51     6]
 [ 1421     7   952